# Code Analysis

The code below is the header for the NLTK implementation of Eliza.
It also imports the needed libraries.

In [8]:
# Natural Language Toolkit: Eliza
#
# Copyright (C) 2001-2023 NLTK Project
# Authors: Steven Bird <stevenbird1@gmail.com>
#          Edward Loper <edloper@gmail.com>
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT

# Based on an Eliza implementation by Joe Strout <joe@strout.net>,
# Jeff Epler <jepler@inetnebr.com> and Jez Higgins <mailto:jez@jezuk.co.uk>.

# a translation table used to convert things you say into things the
# computer says back, e.g. "I am" --> "you are"

from nltk.chat.util import Chat, reflections

Below is a table of response pairs, where the users input is parsed with Regex and potential responses are listed based on what is being said.

The table's format is in: (Regex of phrase to look for, (list of potential responses to input)).

In [ ]:
# a table of response pairs, where each pair consists of a
# regular expression, and a list of possible responses,
# with group-macros labelled as %1, %2.
pairs = (
    # Table format (Regex of phrase to look for, (potential responses))
    (
        r"I need (.*)", # Regex of phrase to look for "I need X"
        (
            "Why do you need %1?", # Probing the necessity of the stated need
            "Would it really help you to get %1?", # Questions the effectiveness of fulfilling the need
            "Are you sure you need %1?", # Challenges the certainty of the need
        ),
    ),
    (
        r"Why don\'t you (.*)",
        (
            "Do you really think I don't %1?",  # Questions the assumption that the program does not perform the action
            "Perhaps eventually I will %1.",  # Implies potential future capability
            "Do you really want me to %1?",  # Inquires about the user's desire for the program to perform the action

        ),
    ),
    (
        r"Why can\'t I (.*)",
        (
            "Do you think you should be able to %1?",  # Asks about the user's belief in their ability
            "If you could %1, what would you do?",  # Encourages the user to consider the implications of being able to perform the action
            "I don't know -- why can't you %1?",  # Reflective question about the user's limitations
            "Have you really tried?",  # Suggests exploring whether the user has attempted the action

        ),
    ),
    (
        r"I can\'t (.*)",
        (
            "How do you know you can't %1?",  # Questions the certainty of the user's inability
            "Perhaps you could %1 if you tried.",  # Suggests potential capability with effort
            "What would it take for you to %1?",  # Inquires about the conditions necessary for the user to perform the action

        ),
    ),
    (
        r"I am (.*)",
        (
            "Did you come to me because you are %1?",  # Investigates the user's motive for stating their current state
            "How long have you been %1?",  # Inquires about the duration of the user's current state
            "How do you feel about being %1?",  # Asks about the user's emotions regarding their current state

        ),
    ),
    (
        r"I\'m (.*)", # Different set of responses for the contraction of "I am". 
        (
            "How does being %1 make you feel?",  # Asks about the user's emotional response to their state
            "Do you enjoy being %1?",  # Questions the user's satisfaction with their current state
            "Why do you tell me you're %1?",  # Inquires about the user's motive for sharing their state
            "Why do you think you're %1?",  # Asks about the user's perception of their state

        ),
    ),
    (
        r"Are you (.*)",
        (
            "Why does it matter whether I am %1?",  # Questions the significance of the program's state
            "Would you prefer it if I were not %1?",  # Asks about the user's preference regarding the program's state
            "Perhaps you believe I am %1.",  # Acknowledges the user's perception of the program's state
            "I may be %1 -- what do you think?",  # Encourages the user to share their opinion on the program's state

        ),
    ),
    (
        r"What (.*)",
        (
            "Why do you ask?",  # Inquires about the user's motive for asking the question
            "How would an answer to that help you?",  # Encourages the user to reflect on the purpose of their question
            "What do you think?",  # Invites the user to share their thoughts or opinions
        ),
    ),
    (
        r"How (.*)",
        (
            "How do you suppose?",  # Encourages the user to speculate or consider possibilities
            "Perhaps you can answer your own question.",  # Suggests self-reflection or problem-solving
            "What is it you're really asking?",  # Encourages the user to clarify their question or intent
        ),
    ),
    (
        r"Because (.*)",
        (
            "Is that the real reason?",  # Questions the validity or depth of the stated reason
            "What other reasons come to mind?",  # Encourages the user to consider alternative explanations
            "Does that reason apply to anything else?",  # Explores the broader implications of the stated reason
            "If %1, what else must be true?",  # Prompts the user to consider additional logical consequences
        ),
    ),
    (
        r"(.*) sorry (.*)",
        (
            "There are many times when no apology is needed.",  # Challenges the necessity of an apology
            "What feelings do you have when you apologize?",  # Encourages the user to reflect on their emotions when apologizing
        ),
    ),
    (
        r"Hello(.*)",
        (
            "Hello... I'm glad you could drop by today.",  # Greets the user warmly
            "Hi there... how are you today?",  # Initiates friendly conversation
            "Hello, how are you feeling today?",  # Inquires about the user's emotional state
        ),
    ),
    (
        r"I think (.*)",
        (
            "Do you doubt %1?",  # Questions the certainty or confidence of the user's statement
            "Do you really think so?",  # Seeks confirmation or clarification of the user's belief
            "But you're not sure %1?",  # Challenges the user's certainty or explores potential doubt
				)
    ),
    (
        r"(.*) friend (.*)",
        (
            "Tell me more about your friends.",  # Invites the user to share information about their social connections
            "When you think of a friend, what comes to mind?",  # Encourages the user to reflect on their perceptions of friendship
            "Why don't you tell me about a childhood friend?",  # Prompts the user to reminisce about past relationships
        ),
    ),
    (
        r"Yes", 
        (
            "You seem quite sure.",  # Acknowledges the user's confidence in their statement
            "OK, but can you elaborate a bit?",  # Requests further explanation or detail
        )
    ),
    (
        r"(.*) computer(.*)",
        (
            "Are you really talking about me?",  # Questions if the user is referring to the program itself
            "Does it seem strange to talk to a computer?",  # Inquires about the user's perception of interacting with a computer
            "How do computers make you feel?",  # Asks about the user's emotional response to computers
            "Do you feel threatened by computers?",  # Explores the user's feelings of insecurity or intimidation towards computers
        ),
    ),
    (
        r"Is it (.*)",
        (
            "Do you think it is %1?",  # Asks for the user's opinion or belief regarding the stated condition
            "Perhaps it's %1 -- what do you think?",  # Encourages the user to consider alternative possibilities
            "If it were %1, what would you do?",  # Prompting the user to consider potential actions based on the stated condition
            "It could well be that %1.",  # Acknowledges the possibility of the stated condition being true
        ),
    ),
    (
        r"It is (.*)",
        (
            "You seem very certain.",  # Acknowledges the user's confidence in the stated condition
            "If I told you that it probably isn't %1, what would you feel?",  # Encourages the user to reflect on their emotions in response to potential contradiction
        ),
    ),
    (
        r"Can you (.*)",
        (
            "What makes you think I can't %1?",  # Asks for the reasoning behind the user's doubt in the program's capability
            "If I could %1, then what?",  # Encourages the user to consider the implications if the program could perform the stated action
            "Why do you ask if I can %1?",  # Inquires about the user's motive for questioning the program's ability
        ),
    ),
    (
        r"Can I (.*)",
        (
            "Perhaps you don't want to %1.",  # Suggests hesitation or reluctance on the user's part to perform the stated action
            "Do you want to be able to %1?",  # Inquires about the user's desire or intention to perform the action
            "If you could %1, would you?",  # Prompting the user to consider hypothetical scenarios regarding their ability to perform the action
        ),
    ),
    (
        r"You are (.*)",
        (
            "Why do you think I am %1?",  # Inquires about the user's perception or reasoning behind attributing a certain characteristic to the program
            "Does it please you to think that I'm %1?",  # Explores the user's emotional response to ascribing certain traits to the program
            "Perhaps you would like me to be %1.",  # Suggests the user may have preferences regarding the program's attributes
            "Perhaps you're really talking about yourself?",  # Suggests that the user's statement may reflect their own thoughts or feelings
        ),
    ),
    (
        r"You\'re (.*)",
        (
            "Why do you say I am %1?",  # Asks for clarification or justification for attributing a certain characteristic to the program
            "Why do you think I am %1?",  # Inquires about the user's perception or reasoning behind the statement
            "Are we talking about you, or me?",  # Prompts the user to consider the subject of the conversation
        ),
    ),
    (
        r"I don\'t (.*)",
        (
             "Don't you really %1?", # Questions the user's assertion, but more importantly
             "Why don't you %1?", # This and the next response encourages further thought and exploration as to WHY the user doesn't do something
             "Do you want to %1?"
        ),
    ),
    (
        r"I feel (.*)",
        (
            "Good, tell me more about these feelings.",  # Encourages the user to elaborate on their emotions
            "Do you often feel %1?",  # Inquires about the frequency of the stated emotion
            "When do you usually feel %1?",  # Asks about the circumstances or triggers associated with the stated emotion
            "When you feel %1, what do you do?",  # Encourages the user to reflect on their coping mechanisms or behaviors in response to the stated emotion
        ),
    ),
    (
        r"I have (.*)",
        (
            "Why do you tell me that you've %1?",  # Inquires about the significance or motive behind sharing the information
            "Have you really %1?",  # Questions the veracity or certainty of the user's statement
            "Now that you have %1, what will you do next?",  # Prompts the user to consider the implications or future actions related to the stated possession
        ),
    ),
    (
        r"I would (.*)",
        (
            "Could you explain why you would %1?",  # Asks for clarification or justification for the stated intention
            "Why would you %1?",  # Inquires about the motive or reasoning behind the stated intention
            "Who else knows that you would %1?",  # Prompts the user to consider others who may be aware of or affected by their intention
        ),
    ),
    (
        r"Is there (.*)",
        (
            "Do you think there is %1?",  # Asks for the user's opinion or belief regarding the existence of the stated condition
            "It's likely that there is %1.",  # Expresses agreement or support for the possibility of the stated condition
            "Would you like there to be %1?",  # Inquires about the user's preference or desire regarding the existence of the stated condition
        ),
    ),
    (
        r"My (.*)",
        (
            "I see, your %1.",  # Acknowledges the user's possession or association with the stated attribute
            "Why do you say that your %1?",  # Inquires about the significance or motive behind the user's statement
            "When your %1, how do you feel?",  # Encourages the user to reflect on their emotions or experiences related to the stated attribute
        ),
    ),
    (
        r"You (.*)",
        (
            "We should be discussing you, not me.",  # Redirects the focus of the conversation back to the user
            "Why do you say that about me?",  # Inquires about the user's reasoning or motive behind their statement regarding the program
            "Why do you care whether I %1?",  # Questions the significance of the user's interest in the program's actions or behaviors
        ),
    ),
    (
        r"Why (.*)", 
        (
            "Why don't you tell me the reason why %1?", # Both responses encourages the user to explain their own rationale or motivation behind their question.
            "Why do you think %1?" # Kinda cheap but I guess it works
        )
    ),
    (
        r"I want (.*)",
        (
            "What would it mean to you if you got %1?",  # Inquires about the significance or importance of the desired outcome to the user
            "Why do you want %1?",  # Encourages the user to explore the motives or desires underlying their want
            "What would you do if you got %1?",  # Prompts the user to consider potential actions or implications associated with obtaining the desired outcome
            "If you got %1, then what would you do?",  # Encourages further reflection on the potential consequences or follow-up actions
        ),
    ),
    (
        r"(.*) mother(.*)",
        ( # It seems any mention of the user mother will be met with a request to elaborate on the user's relationship with their mother
            "Tell me more about your mother.",  # Encourages the user to share additional information about their mother
            "What was your relationship with your mother like?",  # Inquires about the dynamics and interactions within the user's relationship with their mother
            "How do you feel about your mother?",  # Asks about the user's emotions or sentiments regarding their mother
            "How does this relate to your feelings today?",  # Encourages the user to connect their past experiences or relationships with their current emotions
            "Good family relations are important.",  # Provides a general statement acknowledging the significance of positive familial connections
        ),
    ),
    (
        r"(.*) father(.*)",
        ( # Similar to the responses from when the mother is mentioned except for the last
            "Tell me more about your father.",
            "How did your father make you feel?",
            "How do you feel about your father?",
            "Does your relationship with your father relate to your feelings today?",
            "Do you have trouble showing affection with your family?", # Why is this response only in the father? Is it implying that the mention of a father tends to be problematic.
        ),
    ),
    (
        r"(.*) child(.*)",
        (
            "Did you have close friends as a child?",  # Inquires about the user's childhood social experiences
            "What is your favorite childhood memory?",  # Encourages the user to reminisce about positive childhood experiences
            "Do you remember any dreams or nightmares from childhood?",  # Explores the user's recollections of dreams or nightmares from their childhood
            "Did the other children sometimes tease you?",  # Raises the topic of childhood teasing or bullying
            "How do you think your childhood experiences relate to your feelings today?",  # Encourages the user to consider the connection between past experiences and current emotions
        ),
    ),
    (
        r"(.*)\?", # Matches any questions from the user. I was wondering why the above questions (why, how) didn't have question marks in the regex. That means these following responses are potential responses for why and how inputs as well.
        (
            "Why do you ask that?",  # Inquires about the user's motive or intention behind asking the question
            "Please consider whether you can answer your own question.",  # Encourages the user to reflect on whether they already possess the answer to their question
            "Perhaps the answer lies within yourself?",  # Suggests that the user may already know the answer to their question
            "Why don't you tell me?",  # Encourages the user to share their thoughts or insights on the topic of their question
        ),
    ),
    (
        r"quit",
        (
            "Thank you for talking with me.",  # Expresses gratitude for the conversation
            "Good-bye.",  # Offers a farewell
            "Thank you, that will be $150.  Have a good day!",  # Humorous response suggesting a fee for the conversation
        ),
    ),
    (
        r"(.*)", # The responses below are potential for any kind of input from the user. Probably a good catch all in case the use says anything unexpected.
        (
            "Please tell me more.",  # Encourages the user to provide additional information or details
            "Let's change focus a bit... Tell me about your family.",  # Redirects the conversation to a new topic
            "Can you elaborate on that?",  # Requests further explanation or clarification
            "Why do you say that %1?",  # Inquires about the reasoning behind the user's statement
            "I see.",  # Acknowledges the user's input
            "Very interesting.",  # Expresses interest or curiosity
            "%1.",  # Repeats the user's statement, but why include this at all?
            "I see.  And what does that tell you?",  # Encourages the user to reflect on the implications of their statement
            "How does that make you feel?",  # Inquires about the user's emotional response
            "How do you feel when you say that?",  # Asks about the user's emotional state during the conversation

        ),
    ),
)

These input patterns and reponses are arguably rudimentary. There are some questionable responses, like some of the responses assume that the user input something that they feel good about but I can think of many cases where the input is actually a bad feeling.

For example, the matching input is "I feel ___" and the first reponse is "Good, tell me more about these feelings." The assumption is that the input is "I feel (insert a feeling that tends to be positive here)" but there are many cases where if the feeling isn't good, the response would make no sense.

Another response case that peaked my interest was the responses for the mention of father or mother. Most of the responses for both are similar except for one. The difference in this one response is that for the mother, the response is "Good family relations are important" which imply a good relationship, while for the father it is "Do you have trouble showing affection with your family?" Although the response for the mention of a mother doesn't explicitly state it's a good relationship, it's implied that the relationship is good. While the mention of a father directly ask about troubles with the family. Does this different imply anything?

In [ ]:
eliza_chatbot = Chat(pairs, reflections) 

The Chat package is provided by the NLTK library to construct a simple chatbot. The documentation says that these chatbots perform simple pattern matching on user input and respond with automatically generated sentences.

The passed in paramters are `pairs` and `reflections`.

`pairs` is a table of Regex patterns to match user input with potential responses. The `reflections` is another imported module that pair possible phrases in the user input to potential phrases the chatbot can use in its' response. That is, if the user inputs "i am" the chatbot can use "you are" in its response.

Below seems to be the default reflections upon importing:

In [ ]:
reflections = {
    "i am": "you are",
    "i was": "you were",
    "i": "you",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "you are": "I am",
    "you were": "I was",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you",
}

According to the NLTK documentation, the following two functions need to be defined and called. Oddly enough, not much is written about the `demo()` and `eliza_chat()` functions.

The `converse()` function is defined in the NLTK documentation. It starts a loop that effectively holds the conversation with the user. This loops check for the quit condition, otherwise it feeds the user input into a `response()` function which generates a response from the `reflection` and `pairs` parameters fed into the `Chat` moduel.

In [ ]:
def eliza_chat():
    print("Therapist\n---------")
    print("Talk to the program by typing in plain English, using normal upper-")
    print('and lower-case letters and punctuation.  Enter "quit" when done.')
    print("=" * 72)
    print("Hello.  How are you feeling today?")

    eliza_chatbot.converse()

def demo():
    eliza_chat()

This is one way to run the Python code. It's essentially the `main()` method from other programming languages.

In [ ]:
if __name__ == "__main__":
    demo()

# Summary and My Thoughts

I have written a lot of what I thought about this implementation of Eliza throughout the entire notebook. One thing I would like to expand upon is that although I think the implementation is rudimentary, I was still able to have a somewhat deep conversation with it. Shown in the attached screenshots, I talked quite a lot about my sister who is a new international student. We did not grow up together as she grew up in Vietnam with my parents while I lived in the US with my grandparents. The questions and responses from Eliza were enough to get my to explore my thoughts and feelings on my relationship with my sister. Regardless of how simple or rudimentary this program is, it can be effective and I'm totally sold on it. I would have continued the conversation with Eliza about my sister but it seemed to have bugged out and just kept repeating what I was saying. 